In [1]:
"""
This just needs a replacement filepath for list_of_files & list_of_recntfiles depending on the sample file locations
"""


import datetime as dt
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import os
import pandas as pd

%matplotlib

def gimme_dt_nice():
    dowtt = dt.date.today().ctime()  # Day of week to trim
    now = dt.datetime.now() # current date and time
    todayt = dt.date.today()
    todayt = todayt.strftime("%m.%d.%Y")
    ideal_t = dowtt[0:3] + '_' + todayt
    return ideal_t

# list_of_files = glob.glob(r'C:\Users\antho\Downloads\*.csv')
list_of_files = glob.glob(r'C:\Users\antho\IPYNBWork\Sigmoid_Plotter_Sophisticated_Plotting\*.csv')

s = [x for x in list_of_files if 'ExportedTransactions_example' in x]
latest_file = max(s, key=os.path.getctime)
print(latest_file)

df_fromcsv_nu = pd.read_csv(latest_file)

for i in df_fromcsv_nu.columns:
    print(i)

# display(df_fromcsv_nu)

list_of_recntfiles = glob.glob(r'C:\Users\antho\IPYNBWork\Sigmoid_Plotter_Sophisticated_Plotting\*.csv')
srcnt = [x for x in list_of_recntfiles if 'CheckingTxns_example' in x]

latest_filercnt = max(srcnt, key=os.path.getctime)
df_fromcsv = pd.read_csv(latest_filercnt)

ideal_t = gimme_dt_nice()


df_fromcsv = df_fromcsv.append(df_fromcsv_nu, ignore_index=True)
df_fromcsv.to_csv(f'CheckingTxns_{ideal_t}.csv')


df_fromcsv['Posting Date'] = df_fromcsv['Posting Date'].astype('datetime64[ns]')

df_fromcsv = df_fromcsv.sort_values(by='Posting Date', ascending=False)
df_fromcsv = df_fromcsv.reset_index(drop=True)
# display(df_fromcsv)

datey = df_fromcsv['Posting Date'].astype('datetime64[ns]')
blc = df_fromcsv['Balance']
##########
## Starting Values
blc_0 = df_fromcsv.loc[0, 'Balance']
st_dt_pred = df_fromcsv.loc[0, 'Posting Date']

################################################################################
## Date Ranges
start_dt = dt.datetime.strptime('2021-05-01 00:00:00', '%Y-%m-%d %H:%M:%S')
today_dt = dt.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)  
end_dt = today_dt + dt.timedelta(days=int(1.5*365))

predictive_delta = end_dt - start_dt  # Dates from start date to future 90 days
date_list_pred = [end_dt - dt.timedelta(days=x) for x in range(predictive_delta.days)]
df_predict = pd.DataFrame (date_list_pred, columns = ['Date'])

full_datey = df_predict['Date'].astype('datetime64[ns]')
zs = np.zeros(full_datey.shape[0])


df_predict["Routine Expense"] = 0
df_predict = df_predict.sort_values(by='Date', ascending=True)

################################################################################
## Expenses - both income(+) and bills(-)
prd_start_dt = dt.datetime.strptime('2022-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')
st_dt_pred = df_fromcsv.loc[0, 'Posting Date']


df_pf = df_predict[df_predict['Date'] >= prd_start_dt]
df_pf = df_pf.sort_values(by='Date', ascending=True)
df_pf = df_pf.reset_index(drop=True)

# display(df_pf.head(50))


print("Pay Days")
for i, j in df_pf['Date'].items():
    if j.day == 1:
        df_pf.loc[i, 'Routine Expense'] = df_pf.loc[i, 'Routine Expense'] -2044.17
    if j.day ==13:
        df_pf.loc[i, 'Routine Expense'] = df_pf.loc[i, 'Routine Expense'] -175
    modulo = i%14
    if modulo == 0:
        df_pf.loc[i, 'Routine Expense'] = df_pf.loc[i, 'Routine Expense'] + 1339.17
        print(i, j)

# display(df_pf.head(-50))

# display(df_pf)
########################################
########## Will need to be replaced with today's date or most recent of the data
df_pf_postblc = df_pf.loc[df_pf['Date'] >= st_dt_pred]


# display(df_pf_postblc)

blc_0 = df_fromcsv.loc[0, 'Balance']
print(f"Today's Balance: {blc_0}")

indy = df_pf_postblc.loc[df_pf_postblc['Date']== st_dt_pred]  #.index()

balance_indx = indy.index[0]

df_pf_postblc.loc[balance_indx, 'Routine Expense'] = blc_0

# display(df_pf_postblc)

df_pf_postblc['Routine Expense Cumulative Sum'] = df_pf_postblc['Routine Expense'].cumsum()

df_pf_postblc["Routine Minimum"] = 0  # 370, 595, 1718
df_pf_postblc["Routine Average"] = 0
# df_pf_postblc["Routine Maximum"] = 0
df_pf_postblc["Routine Avg No Overdrafts"] = 0
df_pf_postblc["Routine Avg No Overdrafts, Never under 200"] = 0


for idx, txndt in df_pf_postblc['Date'].items():
    dia = txndt.day
    if dia == 14 or dia == 28:
        df_pf_postblc.loc[idx, 'Routine Minimum'] = df_pf_postblc.loc[idx, 'Routine Minimum'] -370/2
        df_pf_postblc.loc[idx, 'Routine Avg No Overdrafts'] = df_pf_postblc.loc[idx, 'Routine Avg No Overdrafts'] -450/2
        
        df_pf_postblc.loc[idx, 'Routine Average'] = df_pf_postblc.loc[idx, 'Routine Average'] -595/2
#         df_pf_postblc.loc[idx, 'Routine Maximum'] = df_pf_postblc.loc[idx, 'Routine Maximum'] -1718/2
        df_pf_postblc.loc[idx, 'Routine Avg No Overdrafts, Never under 200'] = df_pf_postblc.loc[idx, 'Routine Avg No Overdrafts, Never under 200'] -415/2

# df_pf_postblc = df_pf_postblc.rename({"CUMSUM": "Routine Expense Cumulative Sum"}, axis=1)

for rout_col in df_pf_postblc.columns[-4::]:
    print(rout_col)
    titl = f"{rout_col} Expenses"
    df_pf_postblc[titl] = df_pf_postblc['Routine Expense'] + df_pf_postblc[rout_col]
    nutitl = f"{titl} Cumulative Sum"
    df_pf_postblc[nutitl] = df_pf_postblc[titl].cumsum()


# display(df_pf_postblc)

fig, ax = plt.subplots(1, figsize=(15,7))

plot1, = ax.plot(datey, blc, lw=2, label='Balance')

plot2, = ax.plot(full_datey, zs, lw=2, label='Zero', color='red')

dfdatey = df_pf_postblc['Date'].astype('datetime64[ns]')
avgo_595 = df_pf_postblc['Routine Average Expenses Cumulative Sum']
avgo_450 = df_pf_postblc['Routine Avg No Overdrafts Expenses Cumulative Sum']
avgo_415 = df_pf_postblc['Routine Avg No Overdrafts, Never under 200 Expenses Cumulative Sum']
avgo_370 = df_pf_postblc['Routine Minimum Expenses Cumulative Sum']

plot3, = ax.plot(dfdatey, avgo_595, lw=2, label='Average Monthly Surplus of $595 Spent', color = 'darkorange')
plot4, = ax.plot(dfdatey, avgo_450, lw=2, label='No Overdraft Monthly Surplus of $450 Spent', color='gold')
plot5, = ax.plot(dfdatey, avgo_415, lw=2, label='No Overdraft Monthly Surplus of $415 Spent, Balance > 200', color='yellowgreen')
plot6, = ax.plot(dfdatey, avgo_370, lw=2, label='Minimum Monthly Surplus of $370 Spent', color = 'green')


display(df_pf_postblc.head(-90))

leg = ax.legend(fancybox=True, shadow=True)

lines = [plot1, plot2, plot3, plot4, plot5 ,plot6]
lined = {}  # Will map legend lines to original lines.
for legline, origline in zip(leg.get_lines(), lines):
    legline.set_picker(True)  # Enable picking on the legend line.
    legline.set_pickradius(6)
    lined[legline] = origline


def on_pick(event):
    # On the pick event, find the original line corresponding to the legend
    # proxy line, and toggle its visibility.
    legline = event.artist
    origline = lined[legline]
    print(type(origline), origline)
    visible = not origline.get_visible()
    origline.set_visible(visible)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled.
    legline.set_alpha(0.9 if visible else 0.2)
    fig.canvas.draw()


# ax.legend(loc='upper left')
ax.set_xlabel('Transaction Month')
ax.set_ylabel('position')
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
ax.grid()
fig.canvas.mpl_connect('pick_event', on_pick)
plt.show()


C:\Users\antho\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Using matplotlib backend: Qt5Agg
C:\Users\antho\IPYNBWork\Sigmoid_Plotter_Sophisticated_Plotting\ExportedTransactions_example.csv
Unnamed: 0
Posting Date
Balance
Pay Days
0 2022-06-01 00:00:00
14 2022-06-15 00:00:00
28 2022-06-29 00:00:00
42 2022-07-13 00:00:00
56 2022-07-27 00:00:00
70 2022-08-10 00:00:00
84 2022-08-24 00:00:00
98 2022-09-07 00:00:00
112 2022-09-21 00:00:00
126 2022-10-05 00:00:00
140 2022-10-19 00:00:00
154 2022-11-02 00:00:00
168 2022-11-16 00:00:00
182 2022-11-30 00:00:00
196 2022-12-14 00:00:00
210 2022-12-28 00:00:00
224 2023-01-11 00:00:00
238 2023-01-25 00:00:00
252 2023-02-08 00:00:00
266 2023-02-22 00:00:00
280 2023-03-08 00:00:00
294 2023-03-22 00:00:00
308 2023-04-05 00:00:00
322 2023-04-19 00:00:00
336 2023-05-03 00:00:00
350 2023-05-17 00:00:00
364 2023-05-31 00:00:00
378 2023-06-14 00:00:00
392 2023-06-28 00:00:00
406 2023-07-12 00:00:00
420 2023-07-26 00:00:00
434 2023-08-09 00:00:00
448 2023-08-23 00:00:00
462 2023-09-06 00:00:00
476 2023-09-20 00:00:0

<ipython-input-1-096cdb7c3ad2>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fromcsv = df_fromcsv.append(df_fromcsv_nu, ignore_index=True)
<ipython-input-1-096cdb7c3ad2>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pf_postblc['Routine Expense Cumulative Sum'] = df_pf_postblc['Routine Expense'].cumsum()
<ipython-input-1-096cdb7c3ad2>:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pf_postblc["Routin

Routine Avg No Overdrafts
Routine Avg No Overdrafts, Never under 200


Date  Routine Expense  Routine Expense Cumulative Sum  \
216 2023-01-03            92.34                           92.34   
217 2023-01-04             0.00                           92.34   
218 2023-01-05             0.00                           92.34   
219 2023-01-06             0.00                           92.34   
220 2023-01-07             0.00                           92.34   
..         ...              ...                             ...   
671 2024-04-02             0.00                         9658.23   
672 2024-04-03          1339.17                        10997.40   
673 2024-04-04             0.00                        10997.40   
674 2024-04-05             0.00                        10997.40   
675 2024-04-06             0.00                        10997.40   

     Routine Minimum  Routine Average  Routine Avg No Overdrafts  \
216                0              0.0                          0   
217                0              0.0                          0   
218                0              0.0                          0   
219                0              0.0                          0   
220                0              0.0                          0   
..               ...              ...                        ...   
671                0              0.0                          0   
672                0              0.0                          0   
673                0              0.0                          0   
674                0              0.0                          0   
675                0              0.0                          0   

     Routine Avg No Overdrafts, Never under 200  Routine Minimum Expenses  \
216                                         0.0                     92.34   
217                                         0.0                      0.00   
218                                         0.0                      0.00   
219                                         0.0                      0.00   
220                                         0.0                      0.00   
..                                          ...                       ...   
671                                         0.0                      0.00   
672                                         0.0                   1339.17   
673                                         0.0                      0.00   
674                                         0.0                      0.00   
675                                         0.0                      0.00   

     Routine Minimum Expenses Cumulative Sum  Routine Average Expenses  \
216                                    92.34                     92.34   
217                                    92.34                      0.00   
218                                    92.34                      0.00   
219                                    92.34                      0.00   
220                                    92.34                      0.00   
..                                       ...                       ...   
671                                  4108.23                      0.00   
672                                  5447.40                   1339.17   
673                                  5447.40                      0.00   
674                                  5447.40                      0.00   
675                                  5447.40                      0.00   

     Routine Average Expenses Cumulative Sum  \
216                                    92.34   
217                                    92.34   
218                                    92.34   
219                                    92.34   
220                                    92.34   
..                                       ...   
671                                   733.23   
672                                  2072.40   
673                                  2072.40   
674                                  2072.40   
675                                  2072.40  